# Remember to change the File Paths before submitting!!!!!!!!

In [27]:
# ! pip install pandas
# ! pip install scikit-learn
%config Completer.use_jedi = False

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [28]:
# Reading the Training Data

train_data = pd.read_csv("./kaggle/input/titanic/train.csv")
train_data.head()



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [29]:
# Reading the Testing Data

validation_data = pd.read_csv("./kaggle/input/titanic/test.csv")
validation_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [30]:
# Preprocessing removal
# Setup target column
X = train_data.drop("Survived", axis=1) # Removing the target column from the features list
y = train_data["Survived"] # Isolating the target column for y

# Removal of features unwanted for the model to use
# X = X.drop("Ticket", axis=1)
# X = X.drop("Embarked", axis=1)
# X = X.drop("Cabin", axis=1)
# X = X.drop("Fare", axis=1)
# X = X.drop("Name", axis=1)
# X = X.drop("PassengerId", axis=1)

# Designating which columns we want to use to train the model
numerical =     ["Pclass", "Age", "SibSp", "Parch"]
categorical =   ["Sex"]

X.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [31]:
# Creating transformer that normalizes data
numericTransformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),    # Makes all missing values as the mean
    ("scaler", StandardScaler())    # Normalizes everything
])

# Creating transformer that One-Hot Encodes
categoricalTransformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

In [32]:
# Placing everything into a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numericTransformer, numerical),
        ("cat", categoricalTransformer, categorical)
    ]
)

In [33]:
# Putting everything together
classifier = Pipeline(steps=[
    ("preprocessor", preprocessor), # Preprocessing 1st, then
    ("classifier", LogisticRegression(max_iter=1000))   # Doing 1000 iterations of Logistic Regression if we can
])

In [37]:
# Training
classifier.fit(X, y)

# Validation
y_prediction = classifier.predict(validation_data)
y_probability = classifier.predict_proba(validation_data)

output = pd.DataFrame({"PassengerId": test_data.PassengerId, "Survived": y_prediction})
output.to_csv("submission.csv", index=False)